example https://papers.ssrn.com/sol3/papers.cfm?abstract_id=3539694

In [1]:
%run ArticleScraper.ipynb

  Using cached https://files.pythonhosted.org/packages/1a/70/1935c770cb3be6e3a8b78ced23d7e0f3b187f5cbfab4749523ed65d7c9b1/requests-2.23.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/cb/a1/c698cf319e9cfed6b17376281bd0efc6bfc8465698f54170ef60a485ab5d/beautifulsoup4-4.8.2-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/dd/ba/a0e6866057fc0bbd17192925c1d63a3b85cf522965de9bc02364d08e5b84/lxml-4.5.0-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/73/22/a5297f3a1f92468cc737f8ce7ba6e5f245fcfafeae810ba37bd1039ea01c/DateTime-4.3-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/89/e3/afebe61c546d18fb1709a61bee788254b40e736cff7271c7de5de2dc4128/idna-2.9-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/b9/63/df50cac98ea0d5b006c55a399c3bf1db9da7b5a24de7890bc9cfd5dd9e99/certifi-2019.11.28-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org

In [2]:
class SsrnSingleScraper(ArticleScraper):
  page_dict = None

  def __init__(self, page_dict):
    self.page_dict = page_dict
    ArticleScraper.__init__(self, page_dict['link'])

  @property
  def title(self):
    return self.page.head.title.text.split(' by ')[0]

  @property
  def authors(self):
    return self.page.head.title.text.split(' by ')[1].split(" ::")[0]

  @property
  def doi(self):
    return None

  @property
  def abstract(self):
    div = self.page.find("div", {"class": "abstract-text"})
    text = div.find("p").text
    return text

  @property
  def date(self):
    xpath = '//*[@id="selectable"]/text()'
    text = self.tree.xpath(xpath)[0]
    try:
      date = datetime.strptime(text.split(")")[-2].split("(")[-1], '%m/%d/%Y')
    except:
      date = datetime.strptime(text.split(")")[-2].split("(")[-1], '%B %d, %Y')
    return date.strftime(self.date_format)

  @property
  def body(self):
    return None

  @property
  def source(self):
    return "SSRN"

  @property
  def source_impact_factor(self):
    return None

  @property
  def search_keyword(self):
    return None

  @property
  def categories(self):
    return None

  @property
  def licence(self):
    return None

  @property
  def citations(self):
    return None

  @property
  def organization(self):
    return None

  @property
  def keywords(self):
    keywords = self.tree.xpath('//*[@id="maincontent"]/div[2]/div[1]/div[2]/p[2]/text()')[0]
    return ",".join(keywords.split(';'))

  @property
  def references(self):
    return None

  @property
  def link(self):
    return self.page_dict['link']

  @property
  def extralinks(self):
    return self.page_dict['extra_link']
  

In [3]:
example_dicts = [{'authors': 'C You, Y Deng, W Hu, J Sun, Q Lin, F Zhou… - Available at SSRN …, 2020 - papers.ssrn.com',
  'extra_link': 'https://www.medrxiv.org/content/medrxiv/early/2020/02/11/2020.02.08.20021253.full.pdf',
  'extra_link_text': '[PDF] medrxiv.org',
  'link': 'https://papers.ssrn.com/sol3/papers.cfm?abstract_id=3539694',
  'preview': 'Background: The 2019 novel coronavirus (2019-nCoV) outbreak in Wuhan, China has\nattracted world-wide attention. As of February 11, 2020, a total of 44,730 cases of\npneumonia associated with the 2019-nCoV were confirmed by the National Health …',
  'title': 'Estimation of the Time-Varying Reproduction Number of COVID-19 Outbreak in China'},
 {'authors': 'J Gao, P Zheng, Y Jia, H Chen, Y Mao… - Available at SSRN …, 2020 - papers.ssrn.com',
  'extra_link': '',
  'extra_link_text': '',
  'link': 'https://papers.ssrn.com/sol3/papers.cfm?abstract_id=3541120',
  'preview': 'Background: Huge citizens expos social media during a novel coronavirus disease (COVID-\n19) outbroke in Wuhan, China. We assess the prevalence of mental health problems and\nexamine their association with social media exposure. Methods: We conducted a cross …',
  'title': 'Mental Health Problems and Social Media Exposure During COVID-19 Outbreak'},
 {'authors': 'C GU, W Jiang, T Zhao, B Zheng - Available at SSRN 3551006, 2020 - papers.ssrn.com',
  'extra_link': '',
  'extra_link_text': '',
  'link': 'https://papers.ssrn.com/sol3/papers.cfm?abstract_id=3551006',
  'preview': 'The statistics show that the mortality of COVID-19 is 20 times higher than seasonal flu and\nclose to that of Spanish flu, hence it is becoming an absolute priority for every country to take\nefficient measure to limit the transmission of COVID-19. In this short paper, we propose a …',
  'title': 'Mathematical Recommendations to Fight Against COVID-19'},
  ]

In [4]:
class SsrnScraper():
  def parse(self, pages_dict):
    for d in pages_dict:
      yield SsrnSingleScraper(d).parse()  

scraper = SsrnScraper()
for d in scraper.parse(example_dicts):
  for k in d:
    print(k, d[k], sep=": ")
  print()

Title: Estimation of the Time-Varying Reproduction Number of COVID-19 Outbreak in China
Authors: Chong You, Yuhao Deng, Wenjie Hu, Jiarui Sun, Qiushi Lin, Feng Zhou, Cheng Heng Pang, Yuan Zhang, Zhengchao Chen, Xiao-Hua Zhou
DOI: None
Abstract: Background: The 2019 novel coronavirus (2019-nCoV) outbreak in Wuhan, China has attracted world-wide attention. As of February 11, 2020, a total of 44,730 cases of pneumonia associated with the 2019-nCoV were confirmed by the National Health Commission (NHC) of China.    Methods: Three approaches, namely Poisson likelihood-based method (ML), exponential growth rate-based method (EGR) and stochastic Susceptible-Infected-Removed dynamic model-based method (SIR), were implemented to estimate the basic and controlled reproduction numbers.    Results: A total of 71 chains of transmission together with dates of symptoms onset and 67 dates of infections were identified among 5,405 confirmed cases outside Hubei as reported by February 2, 2020. Based on 

IndexError: list index out of range